In [1]:
import numpy as np

Define problem variables.

In [2]:
M = 3 # number of cells
K = 3 # number of users
rho = 1 # max power
BW_max = 6 # max BW, M_hat in paper
eta = np.random.rand(K) # power control fraction for each user
alpha = np.random.rand(M,K)
beta = np.random.rand(M,K)

In [3]:
#tg = 10.0 ** np.array([-3.2, -3, -2.8, -2.6, -2.4, -2.2, -2., -1.8, -1.6, -1.4, -1.2, -1, -0.8, -0.6, -0.4, -0.2, 0, 1, 2, 3])
tg = 10.0 ** np.array([ -3, -2.5, -2., -1.5, -1, -0.5, 0])
Nt = len(tg)
x_t = np.zeros(Nt)

X_mat = np.zeros((M,K))
x_vec = X_mat.flatten('F')
x = np.concatenate((x_vec,x_t))

In [4]:
Pt = np.concatenate((np.zeros((Nt,M*K)),np.eye(Nt)), axis = 1)
Pk_list = []
for k in range(K):
    Pk = np.concatenate((np.zeros((M,M*k)), np.eye(M), np.zeros((M,M*(K-k-1)+Nt))), axis = 1 )
    Pk_list.append(Pk)
Pm_list = []
for m in range(M):
    Pm = np.zeros((K,M*K+Nt))
    for k in range(K):
        Pm[k,m+M*k] = 1
    Pk_list.append(Pm)
    
one_vec = np.ones(M*K+Nt)
one_vec_tilde = np.concatenate((np.ones(M*K),np.zeros(Nt)))

TO DOs:
- 1) create a function that evaluate SINRs for a given configuration of X and a given eta
- 2) brute force (48) 
- 3) brute force the full problem, i.e. find SINRs for each configuration, find min SINR, search for max min SINR

In [5]:
def SINR( X, eta, alpha, beta, rho ):
    
    K = X.shape[1]
    M = X.shape[0]
    
    snr = np.zeros(K)
    
    for k in range(K):
        num = rho * eta[k]*( np.sum(np.multiply(X[:,k],alpha[:,k]))**2 )
        den = 0
        for k1 in list(range(k))+list(range(k+1,K)):
            den += rho*( eta[k1]* ( np.sum( np.multiply(np.multiply(X[:,k],alpha[:,k]), beta[:,k1] ) ) ) )
        den += np.sum(np.multiply(X[:,k],alpha[:,k]))
        snr[k] = num/den
        if snr[k] != snr[k] : snr[k] = 0    
    
    return snr

Bruteforce the original problem. Gnerate all valid matrices of link combination and compute SINR for each of them.

In [6]:
n_combin = 2**(M*K)

X_list_try = []

for i in range(n_combin):
    x_vec = np.zeros(M*K)
    bs = format(i, "0"+str(M*K)+"b")
    for j in range(M*K): 
        x_vec[j] = bs[j]
    X_list_try.append(x_vec.reshape(M,K))
    
X_list = []
for Xmat in X_list_try:
    if np.sum(Xmat.flatten()) <= BW_max:
        X_list.append(Xmat)

print(len(X_list_try))
print(len(X_list))

512
466


In [7]:
# try a string

x_vec = np.zeros(M*K)
x_mat = np.array([[1,0,1],[0,1,1],[0,0,1]]) # set it to a matrix
print(SINR(x_mat, eta, alpha, beta, rho))

[0.07752993 0.32541509 0.90041286]


In [8]:
min_snr = np.zeros(len(X_list))
for i in range(len(X_list)):
    Xmat = X_list[i]
    snr = SINR(Xmat, eta, alpha, beta, rho)
    min_snr[i] = np.amin(snr)

best_id = np.argmax(min_snr)
best_X = X_list[best_id]

print('Actual full search best\n')
print(best_id)
print(min_snr[best_id])
print(best_X)


Actual full search best

63
0.5111566989681892
[[0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 1.]]


/home/alb/Scrivania/Quantum/Projects/Applications/QAOA-CellFreeMIMO/virtual_QAOA-CellFreeMIMO/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


Now lets' bruteforce (48) and check the two solutions coincide.

In [9]:
print(Pt.shape)
print(tg.shape)

(7, 16)
(7,)


In [10]:
# build A and B

A_list = []
B_list = []

b_tilde_list = []
A_tilde_list = []

for k in range(K):
    A_tilde = np.zeros((M,M))
    b_tilde = np.zeros(M)

    for m in range(M):
        for n in range(M):
            A_tilde[m,n] = rho*eta[k]*alpha[m,k]*alpha[n,k]
        b_tilde[m] = alpha[m,k]*(rho*(np.sum(np.multiply(eta,beta[m,:])) - beta[m,k]*eta[k]) + 1)
    
    A_tilde_list.append(A_tilde)
    b_tilde_list.append(b_tilde)
    
    A_list.append( Pk_list[k].T@A_tilde@Pk_list[k] )
    B_list.append( np.outer((Pt.T @ tg),(b_tilde.T @ Pk_list[k])) )
    
    

In [11]:
n_combin = 2**(M*K+Nt)

print(n_combin)

65536


In [12]:
x_list_try = []

for i in range(n_combin):
    x = np.zeros(M*K + Nt)
    bs = format(i, "0"+str(M*K+Nt)+"b")
    for j in range(M*K+Nt): 
        x[j] = bs[j]
    x_list_try.append(x)
    
x_list = []
for x in x_list_try:
    c1 = np.zeros(K)
    c2 = np.sum(x[:M*K])
    c3 = np.zeros(K)
    oneM = np.ones(M)
    for k in range(K):
        c1[k] = x.T@ ( A_list[k] - B_list[k] ) @ x
        c3[k] = oneM @  Pk_list[k] @ x
        
    if (c1>=0).all() and c2 <= BW_max and (c3>=1).all():
        x_list.append(x)

print(len(x_list_try))
print(len(x_list))

65536
9850


In [13]:
obj_vec = np.zeros(len(x_list))
for i in range(len(x_list)):
    x = x_list[i]
    obj_vec[i] = tg.T @ Pt @ x # NOTE: this is just a lower bound for SNR!!
    
best_id = np.argmax(obj_vec)
best_x = x_list[best_id]


print('New parametrization full search best\n')
print(best_id)
print(obj_vec[best_id])
print(best_x)

New parametrization full search best

3705
0.46201282027869006
[0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0.]


In [21]:
# actual SNR 
bestx_mat = best_x[:M*K].reshape(M,K).T
print(bestx_mat)
print('Actual SINR:\n')
print(SINR(bestx_mat, eta, alpha, beta, rho))

[[0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 1.]]
Actual SINR:

[0.5111567  0.64527657 0.6011947 ]


In [15]:
for k in range(K):
    x = x_list[255]
    b1 = x.T @ B_list[k] @ x
    print(b1)
    b2 = (tg.T @ Pt @ x ).T 
    b2 = b2 *( b_tilde_list[k].T @ Pk_list[k] @ x )
    print(b2)

    a1 = x.T @ A_list[k] @ x
    print(a1)

0.17999832379653616
0.17999832379653616
0.3133954604539209
0.15929596153899145
0.15929596153899145
0.3555735562911831
0.1627082556779938
0.1627082556779938
0.4678180325832295


In [16]:
[print(Pk_list[k] @ x) for k in range(K)]

[0. 0. 1.]
[0. 1. 0.]
[0. 0. 1.]


[None, None, None]

In [19]:
x = best_x
c1 = np.zeros(K)
c2 = np.sum(x[:M*K])
c3 = np.zeros(K)
oneM = np.ones(M)
for k in range(K):
    c1[k] = x.T@ ( A_list[k] - B_list[k] ) @ x
    c3[k] = oneM @  Pk_list[k] @ x
    
print(c3)

[2. 2. 2.]


In [20]:
Pk_list[1]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])